# Orbit Computation

This tutorial demonstrates how to generate satellite orbits using various models.

## Setup

In [ ]:
import numpy as np
import pandas as pd

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected = True)

In [1]:
import Library.Physics as Physics
import Library.Astrodynamics as Astrodynamics

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for library::physics::time::DateTime already registered; second conversion method ignored.
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for library::math::obj::IntervalBase::Type already registered; second conversion method ignored.
  return f(*args, **kwds)


In [2]:
Length = Physics.Units.Length
Angle = Physics.Units.Angle
Time = Physics.Time
LLA = Physics.Coordinate.Spherical.LLA
Frame = Physics.Coordinate.Frame
Environment = Physics.Environment
Earth = Physics.Environment.Objects.CelestialBodies.Earth

Trajectory = Astrodynamics.Trajectory
Orbit = Trajectory.Orbit

---

## SGP4

### Computation

In [3]:
environment = Environment.Default()

Create a Classical Orbital Element (COE) set:

In [4]:
a = Length.Kilometers(7000.0)
e = 0.0001
i = Angle.Degrees(35.0)
raan = Angle.Degrees(40.0)
aop = Angle.Degrees(45.0)
nu = Angle.Degrees(50.0)

coe = Orbit.Models.Kepler.COE(a, e, i, raan, aop, nu)

Setup a Keplerian orbital model:

In [5]:
epoch = Time.Instant.DateTime(Time.DateTime(2018, 1, 1, 0, 0, 0), Time.Scale.UTC)
earth = environment.accessCelestialObjectWithName("Earth")

keplerian_model = Orbit.Models.Kepler(coe, epoch, earth, Orbit.Models.Kepler.PerturbationType.No)

Create a Two-Line Element (TLE) set:

In [6]:
tle = Orbit.Models.SGP4.TLE("1 39419U 13066D   18260.77424112  .00000022  00000-0  72885-5 0  9996",
                            "2 39419  97.6300 326.6556 0013847 175.2842 184.8495 14.93888428262811")

Setup a SGP4 orbital model:

In [7]:
sgp4_model = Orbit.Models.SGP4(tle)

Setup the orbit:

In [8]:
orbit = Orbit(keplerian_model, environment.accessCelestialObjectWithName("Earth"))
# orbit = Orbit(sgp4_model, environment.accessCelestialObjectWithName("Earth"))

In [22]:
model = orbit.accessModel()

In [23]:
type(model)

Library.Astrodynamics.Trajectory.Orbit.Models.Kepler

In [24]:
model.isDefined()

ArgumentError: Python argument types in
    Kepler.isDefined(Kepler)
did not match C++ signature:
    isDefined(library::astro::trajectory::orbit::models::Kepler {lvalue})

In [12]:
model.asSGP4()

ArgumentError: Python argument types in
    OrbitModel.asSGP4(Kepler)
did not match C++ signature:
    asSGP4(library::astro::trajectory::orbit::Model)

Now that the orbit is set, we can compute the satellite position:

In [ ]:
start_instant = Time.Instant.DateTime(Time.DateTime(2018, 9, 5, 0, 0, 0), Time.Scale.UTC)
end_instant = Time.Instant.DateTime(Time.DateTime(2018, 9, 6, 0, 0, 0), Time.Scale.UTC)

In [ ]:
interval = Time.Interval.Closed(start_instant, end_instant)

In [ ]:
step = Time.Duration.Minutes(1.0)

Generate a time grid:

In [ ]:
instants = interval.generateGrid(step)

In [ ]:
states = [[instant, orbit.getStateAt(instant)] for instant in instants]

In [ ]:
def convert_state (instant, state):
    
    lla = LLA.Cartesian(state.getPosition().inFrame(Frame.ITRF(), state.getInstant()).getCoordinates(), Earth.EquatorialRadius, Earth.Flattening)
    
    return [
                repr(instant),
                float(instant.getModifiedJulianDate(Time.Scale.UTC)),
                *state.getPosition().getCoordinates().transpose()[0].tolist(),
                *state.getVelocity().getCoordinates().transpose()[0].tolist(),
                float(lla.getLatitude().inDegrees()),
                float(lla.getLongitude().inDegrees()),
                float(lla.getAltitude().inMeters())
            ]

In [ ]:
orbit_data = [convert_state(instant, state) for [instant, state] in states]

In [ ]:
orbit_df = pd.DataFrame(orbit_data, columns=['$Time^{UTC}$', '$MJD^{UTC}$', '$x_{x}^{ECI}$', '$x_{y}^{ECI}$', '$x_{z}^{ECI}$', '$v_{x}^{ECI}$', '$v_{y}^{ECI}$', '$v_{z}^{ECI}$', '$Latitude$', '$Longitude$', '$Altitude$'])

In [ ]:
orbit_df.head()

### Output

In **Earth Fixed** frame:

In [ ]:
contours = []

contours.append( dict(
    type = 'scattergeo',
    lon = orbit_df['$Longitude$'],
    lat = orbit_df['$Latitude$'],
    mode = 'lines',
    line = dict(
        width = 2,
        color = 'rgb(255, 62, 79)'
    )
) )

layout = dict(
        title = None,
        showlegend = False,
        height=1000,
        geo = dict(
            showland = True,
            showlakes = True,
            showcountries = False,
            showocean = True,
            countrywidth = 0.0,
            landcolor = 'rgb(100, 100, 100)',
            lakecolor = 'rgb(240, 240, 240)',
            oceancolor = 'rgb(240, 240, 240)',
            projection = dict( 
                type = 'orthographic',
                rotation = dict(
                    lon = -100,
                    lat = 40,
                    roll = 0
                )            
            ),
            lonaxis = dict( 
                showgrid = True,
                gridcolor = 'rgb(102, 102, 102)',
                gridwidth = 0.5
            ),
            lataxis = dict( 
                showgrid = True,
                gridcolor = 'rgb(102, 102, 102)',
                gridwidth = 0.5
            )
        )
    )
    
fig = dict(data=contours, layout=layout)

iplot(fig)

In [ ]:
data = []

data.append(
    dict(
        type = 'scattergeo',
        lon = orbit_df['$Longitude$'],
        lat = orbit_df['$Latitude$'],
        mode = 'lines',
        line = dict(
            width = 1,
            color = 'red',
        )
    )
)
    
layout = dict(
        title = None,
        showlegend = False,
        height=1000,
        geo = dict(
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
        ),
    )
    
fig = dict(data=data, layout=layout)
iplot(fig)

In **Earth Inertial** frame:

In [ ]:
theta = np.linspace(0, 2 * np.pi, 30)
phi = np.linspace(0, np.pi, 30)

theta_grid, phi_grid = np.meshgrid(theta, phi)

r = float(Physics.Environment.Objects.CelestialBodies.Earth.EquatorialRadius.inMeters())

x = r * np.cos(theta_grid) * np.sin(phi_grid)
y = r * np.sin(theta_grid) * np.sin(phi_grid)
z = r * np.cos(phi_grid)

earth = go.Surface(
    x=x,
    y=y,
    z=z,
    colorscale='Viridis',
    showscale=False
)

trace = go.Scatter3d(
    x=orbit_df['$x_{x}^{ECI}$'],
    y=orbit_df['$x_{y}^{ECI}$'],
    z=orbit_df['$x_{z}^{ECI}$'],
    mode='lines',
    marker=dict(
        size=0,
        color=orbit_df['$x_{z}^{ECI}$'],
        colorscale='Viridis',
        showscale=False
    ),
    line=dict(
        color=orbit_df['$x_{z}^{ECI}$'],
        width=1
    )
)

data = [earth, trace]

layout = go.Layout(
    title=None,
#     width=800,
    height=1000,
#     autosize=False,
    showlegend=False,
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        camera=dict(
            up=dict(
                x=0,
                y=0,
                z=1
            ),
            eye=dict(
                x=-1.7428,
                y=1.0707,
                z=0.7100,
            )
        ),
        aspectratio = dict( x=1, y=1, z=1 ),
        aspectmode = 'manual'
    ),
)

fig = dict(data=data, layout=layout)

iplot(fig)

Scatter plot:

In [ ]:
plot_data = [
    go.Scatter(
        x=orbit_df['$MJD^{UTC}$'],
        y=orbit_df['$x_{x}^{ECI}$']
    ),
    go.Scatter(
        x=orbit_df['$MJD^{UTC}$'],
        y=orbit_df['$x_{y}^{ECI}$']
    ),
    go.Scatter(
        x=orbit_df['$MJD^{UTC}$'],
        y=orbit_df['$x_{z}^{ECI}$']
    )
]

iplot(plot_data)